## Example 13 - Path selection by vessel limits

In [1]:
# package(s) related to time, space and id
import datetime, time
import platform
import random
import os
import pathlib
import warnings

# you need these dependencies (you can get these from anaconda)
# package(s) related to the simulation
import simpy

# spatial libraries 
import pyproj
import shapely.geometry
from simplekml import Kml, Style

# package(s) for data handling
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# OpenTNSIM
import opentnsim
import opentnsim.core as core
import opentnsim.graph_module as graph_module
import opentnsim.plot as plot
import opentnsim.model as model

# Used for mathematical functions
import math             

# Used for making the graph to visualize our problem
import networkx as nx  

from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 


In [14]:
src_dir = pathlib.Path(opentnsim.__file__).parent.parent

# Graph location
location_graph = src_dir / "notebooks"
name_graph = location_graph / "Shape-Files" / "Vaarwegvakken" / "Vaarwegvakken.shp"

# Vessel database
location_vessel_database = location_graph / "Vessels" / "richtlijnen-vaarwegen-2017.csv"

### Create graph

The cel below visualizes the problem. In graph theory the red dots are called *edges* and the lines are called *vertices*. Vessels (or any other object) move from edge 1 to edge 3 and from edge 4 to edge 2. The added complexity is that vertice 5-6 only allows traffic in one direction at a time. Vessels can travel simultanously in one direction.

**Important**: 

If you use windows and get the following error "ImportError: read_shp requires OGR: http://www.gdal.org/", you probably have [this issue](https://github.com/conda-forge/gdal-feedstock/issues/219). Solving it is possible by running the following commands in your terminal (as explained [here](https://gis.stackexchange.com/questions/294231/installing-gdal-with-anaconda)):

```bash
#Create a new virtual environment
conda create -n testgdal -c conda-forge gdal vs2015_runtime=14

#Activate virtual environment
activate testgdal

#Open Jupyter notebook
jupyer notebook
```

In [15]:
graph = graph_module.Graph()
graph.from_shape(location_graph, name_graph)

In [ ]:
graph.create_graph_new_projection()
graph.plot()

We'll simulate vessels travelling from Utrecht to 's-Hertogenbosch. Given the properties of the graph the origin will be *Rijkshaven Utrecht* and the destination will be *Industriehavens 's-Hertogenbosch*. Resulting in the following nodes:

- Origin      (5.102283, 52.045830)
- Destination (5.289868, 51.698449)

The fasted route will be through the *Kanaal van Sint Andries*, but there is a lock in this canal. For this example we will not simulate the lock, but we will add restrictions to the canal so that not all vessesls can pass through. The edges we'll add restrictions to are between the following nodes. 

- Node 1 (5.368229, 51.791797)
- Node 2 (5.369157, 51.789972)

In [ ]:
for i, edge in enumerate(graph.graph.edges(data = True)):
    if edge[0] == "(5.368229, 51.791797)" and edge[1] == "(5.369157, 51.789972)": 
        edge[2]["Height"] = 5.10
        edge[2]["Width"]  = 9.50
        edge[2]["Depth"]  = 3.50
    
    elif edge[1] == "(5.368229, 51.791797)" and edge[0] == "(5.369157, 51.789972)": 
        edge[2]["Height"] = 5.10
        edge[2]["Width"]  = 9.50
        edge[2]["Depth"]  = 3.50
        
    else:
        edge[2]["Height"] = 30
        edge[2]["Width"]  = 30
        edge[2]["Depth"]  = 30

### Create vessel database

In [ ]:
vessel_db = pd.read_csv(os.path.join(location_graph, location_vessel_database))
vessel_db['v'] = 1
vessel_db['type'] = vessel_db['vessel_type']
vessel_db['B'] = vessel_db['width']
vessel_db['L'] = vessel_db['length']
vessel_db.head()

In [ ]:
# Vessel type
Vessel = type('Vessel', 
              (core.Identifiable, core.Movable, core.HasContainer,
               core.VesselProperties, core.HasResource, core.Routeable,
               core.ExtraMetadata
              ), 
              {})

In [ ]:
generator = model.VesselGenerator(Vessel, vessel_db, loaded = "Random")

### Run simulation

In [ ]:
# Start simpy environment
simulation_start = datetime.datetime(2019, 1, 1)
sim = model.Simulation(simulation_start, graph.graph)

In [ ]:
# Create a random path over the graph
sim.add_vessels(vessel_generator = generator, origin = "(5.102283, 52.045830)", destination = "(5.289868, 51.698449)")

In [ ]:
%%time
# Run the simulation for 100 days
sim.run(duration = 100 * 24 * 60 * 60)

### Check results

In [12]:
# The different routes that are sailed by various vessels
sim.environment.routes

,Origin,Destination,Width,Height,Depth,Route


In [13]:
# The unique routes that are sailed by various vessels
for i, route in enumerate(np.unique(sim.environment.routes["Route"].values)):
    print("Route", i + 1)
    print(route)
    print("")